In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(101)

In [3]:
transform = transforms.ToTensor()

Import the MNIST Dataset

In [4]:
train_data = datasets.MNIST(root = "train_images/MNIST", train=True, download=True, transform=transform)

In [5]:
test_data = datasets.MNIST(root = "train_images/MNIST", train=False, download=True, transform=transform)

In [6]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: train_images/MNIST
    Split: Test
    StandardTransform
Transform: ToTensor()

In [7]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: train_images/MNIST
    Split: Train
    StandardTransform
Transform: ToTensor()

In [8]:
train_loader = DataLoader(dataset=train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=10, shuffle=False)

*conv1 = nn.Conv2d(1,6,3,1)*:

- 1, for one input channel (grayscale)
- 6, output channels for feature extractions, the feature maps, the filters that the CNN will figure out
- 3, kernel size for a 3 by 3 filter
- 1, for the horizontal and vertical stride of the convolution


In [9]:
conv1 = nn.Conv2d(1,6,3,1)
conv2 = nn.Conv2d(6,16,3,1)

In [10]:
for i, (X_train, Y_train) in enumerate(train_data):
    break

In [11]:
X_train.shape

torch.Size([1, 28, 28])

In [12]:
x = X_train.view(1,1,28,28)

In [13]:
out = conv1(x)

In [14]:
out.shape

torch.Size([1, 6, 26, 26])

In [15]:
max_p = F.max_pool2d(out,2,2)

In [16]:
max_p.shape

torch.Size([1, 6, 13, 13])

How to feed this output of max_p to a fully conected input?

In [17]:
x = max_p.view(-1, 6*13*13)

In [18]:
x.shape

torch.Size([1, 1014])

In [19]:
class ConvolutionalNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,6,3,1)
        self.conv2 = nn.Conv2d(6,16,3,1)
        self.fc1 = nn.Linear(5*5*16, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, X):
        X = F.relu(self.conv1(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, 2, 2)
        X = X.view(-1, 16*5*5)
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = self.fc3(X)
        return F.log_softmax(X, dim=1)

In [20]:
model = ConvolutionalNN()
model.to(device)
model

ConvolutionalNN(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [21]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [23]:
import time
start_time = time.time()

epochs = 5
train_loss = []
test_losses = []
train_correct = []
test_correct = []

for epoch in range(epochs):
    trn_corr = 0
    tst_corr = 0
    
    # train 
    for b, (X_trn, Y_trn) in enumerate(train_loader):
        X_trn, Y_trn = X_trn.to(device), Y_trn.to(device)  # Ensure inputs and targets are on the same device
        b += 1
        
        y_pred = model(X_trn)  # Fixed typo: use X_trn instead of X_train
        loss = criterion(y_pred, Y_trn)
        
        predicted = torch.max(y_pred.data, 1)[1]
        batch_corr = (predicted == Y_trn).sum()  # Corrected from y_pred to Y_trn
        trn_corr += batch_corr
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if b % 600 == 0:
            print(f"Epoch {epoch} Batch {b}/ Loss: {loss.item():.4f}")
    train_loss.append(loss.item())
    train_correct.append(trn_corr)  # Use trn_corr instead of batch_corr for the entire epoch
        
    # test
    with torch.no_grad():
        for b, (X_test, Y_test) in enumerate(test_loader):
            X_test, Y_test = X_test.to(device), Y_test.to(device)  # Ensure test inputs and targets are on the same device
            y_val = model(X_test)
            predicted = torch.max(y_val.data, 1)[1]
            tst_corr += (predicted == Y_test).sum()
        
    loss = criterion(y_val, Y_test)
    test_losses.append(loss.item())
    test_correct.append(tst_corr)
        
current_time = time.time()
total = current_time - start_time
print(f"Total time: {total}")

Epoch 0 Batch 600/ Loss: 0.0575
Epoch 0 Batch 1200/ Loss: 0.1789
Epoch 0 Batch 1800/ Loss: 0.0510
Epoch 0 Batch 2400/ Loss: 0.0205
Epoch 0 Batch 3000/ Loss: 0.7319
Epoch 0 Batch 3600/ Loss: 0.1269
Epoch 0 Batch 4200/ Loss: 0.4072
Epoch 0 Batch 4800/ Loss: 0.0064
Epoch 0 Batch 5400/ Loss: 0.0023
Epoch 0 Batch 6000/ Loss: 0.0077
Epoch 1 Batch 600/ Loss: 0.0449
Epoch 1 Batch 1200/ Loss: 0.0094
Epoch 1 Batch 1800/ Loss: 0.1434
Epoch 1 Batch 2400/ Loss: 0.0858
Epoch 1 Batch 3000/ Loss: 0.0198
Epoch 1 Batch 3600/ Loss: 0.0014
Epoch 1 Batch 4200/ Loss: 0.0020
Epoch 1 Batch 4800/ Loss: 0.1435
Epoch 1 Batch 5400/ Loss: 0.0076
Epoch 1 Batch 6000/ Loss: 0.0045
Epoch 2 Batch 600/ Loss: 0.0137
Epoch 2 Batch 1200/ Loss: 0.0014
Epoch 2 Batch 1800/ Loss: 0.2535
Epoch 2 Batch 2400/ Loss: 0.0041
Epoch 2 Batch 3000/ Loss: 0.0059
Epoch 2 Batch 3600/ Loss: 0.0006
Epoch 2 Batch 4200/ Loss: 0.0014
Epoch 2 Batch 4800/ Loss: 0.2201
Epoch 2 Batch 5400/ Loss: 0.0006
Epoch 2 Batch 6000/ Loss: 0.0005
Epoch 3 Batch